# Install requirements via pip etc.

In [ ]:
run_in = 'local'
#run_in = 'colab'

if run_in == 'local':
    import pandas as pd
    import os
    import bs4 as BeautifulSoup
    import requests
    import json
    import numpy as np
    from urllib.parse import urlparse
    import pprint
    pp = pprint.PrettyPrinter(indent=4)
    import html2text as html2text
    import datetime
    import re
    from lxml import html
      
if run_in == 'colab':
    !pip install beautifulsoup4
    !pip install requests
    !pip install html2text
    import pandas as pd
    import os
    import bs4 as BeautifulSoup
    import requests
    import json
    import numpy as np
    from urllib.parse import urlparse
    import pprint
    pp = pprint.PrettyPrinter(indent=4)
    import html2text as html2text
    import datetime
    import re
    from lxml import html

In [ ]:
if run_in == 'local':
  base_path = "/users/USERNAME/Google Drive/USERNAME/"
if run_in == 'colab':
  #base_path = "/content/drive/My Drive/USERNAME/"
  base_path = "/content/drive/My Drive/USERNAME/"
  from google.colab import drive
  drive.mount('/content/drive')
  
print('base_path set to:')
print(base_path)

##<p><font color="red">Define Version number!</font></p>

In [ ]:
version_id = 'V14'

### Check that we can access the GDrive and your personal folder

Make sure that we can see the two things:
1. A file called startups.csv with one column that lists all the twitter handles
2. A folder called 'outputs' in which this script will store all results and log files

In [ ]:
os.listdir(base_path)

###Read all start-ups and their website URLs

In [ ]:
df_domain_dict = pd.read_excel(base_path + "Pitchbook_Crunchbase_Tracxn_Raw/Don't touch " + version_id + "/" + version_id + ".xlsx", version_id + "website_url_scrape", header=None, skiprows=1, names=['company_name','domain'], usecols="A,B")
#df_domain_dict = df_domain_dict[-20:]
df_domain_dict[:10]

In [ ]:
df_domain_dict.shape

In [ ]:
df_domain_dict = df_domain_dict[df_domain_dict.domain != 'NotFunctioning']
df_domain_dict = df_domain_dict.drop_duplicates('domain')

In [ ]:
companies = df_domain_dict["company_name"]
domains = df_domain_dict["domain"]

company2domain = {}
domain2company = {}
for company_name, domain in zip(companies,domains):
    company2domain[company_name] = domain
    domain2company[domain] = company_name
    
print(company2domain)
print(domain2company)

###Check for duplicates

In [ ]:
duplicate_domains = df_domain_dict[df_domain_dict.duplicated()]
print(duplicate_domains)

In [ ]:
list_of_domains = df_domain_dict['domain'].values.tolist()
print(list_of_domains)


#list_of_domains = ['prowler.io', 'google.com','basdlkf.com']
#print(list_of_domains)

In [ ]:
for key in company2domain:
  print(key)

In [ ]:
list_of_scrapable_domains = []
list_of_broken_domains = []

print('\033[1m' + 'request status_codes:' '\033[0m')

result_dictionary = {}

for k in list_of_domains[:]:
   
    try:
        result = requests.get('http://' + k)
        print(result)
        if result.status_code == 200:
            print('http://' + str(k) + ' is ' + str(result.status_code) + ' therefore ' + '\033[1m' + str(k) + '\033[0m' + ' is added to list_of_scrapable_domains')
            list_of_scrapable_domains.append(k)

            companyname = domain2company[k]
            global_pagerank = []
            inbound_links = []
            top_search_keywords = []

            source = requests.get('https://www.alexa.com/siteinfo/' + k)
            html_content = source.text     

            # find global_pagerank
            pattern_global_pagerank = re.compile(r'(?<=\"global\":)(.*)(?=},)')
            matches_global_pagerank = pattern_global_pagerank.finditer(html_content)
            for match in matches_global_pagerank:
              if match.group(0).replace(',', '') == "false":
                global_pagerank.append(np.nan)
              else:
                global_pagerank.append(int(match.group(0).replace(',', '')))


            # findinbound_links
            pattern_inbound_links = re.compile(r'(?<=box1-r\">)(.*)(?=</span>)')
            matches_inbound_links = pattern_inbound_links.finditer(html_content)
            for match in matches_inbound_links:
              if match.group(0).replace(',', '') == "false":
                inbound_links.append(np.nan)
              else:
                inbound_links.append(int(match.group(0).replace(',', '')))

            # find top_search_keywords
            pattern_top_search_keywords = re.compile(r"(?<=llipsis\' title=\')(.*)(?=\'><span)")
            matches_top_search_keywords = pattern_top_search_keywords.finditer(html_content)
            for match in matches_top_search_keywords:
              top_search_keywords.append(match.group(0))


            webscrape_domain = {
            'domain': k,
            'global_pagerank': global_pagerank,
            'inbound_links': inbound_links,
            'top_search_keywords':top_search_keywords
            }
            result_dictionary[companyname] = webscrape_domain

        else:
             print(str(k) + ' is ' + str(result.status_code))
             list_of_broken_domains.append(k)
    except: # requests.exceptions.ConnectionError:
        print("Connection refused by" + str(k))
        list_of_broken_domains.append(k)

print('\n' '\033[1m' + 'list_of_scrapable_domains:' '\033[0m', end =' ')
print(str(list_of_scrapable_domains) + '\n')    

print('\033[1m' + 'list_of_broken_domains:' '\033[0m', end =" ")
print(list_of_broken_domains)

In [ ]:
pp.pprint(result_dictionary)

In [ ]:
with open(base_path + 'outputs/Alexa Page Rank/page_ranks.json', "w", encoding='utf8') as result_file:
    #result_file.write(str(result_dictionary))
    json.dump(result_dictionary, result_file, sort_keys=True, indent=4)

with open(base_path + 'outputs/Alexa Page Rank/broken_website_links.txt', "w", encoding='utf8') as broken_links_file:
    broken_links_file.writelines(["%s\n" % result for result in list_of_broken_domains])